INSTALL STUFF

In [ ]:
!pip install biopython
!pip install torch
!pip install tape_proteins
!pip install Bio
!pip install import-ipynb
!pip install git+https://github.com/facebookresearch/esm.git

IMPORT STUFF

In [ ]:
import pathlib
import torch
from Bio.PDB import PDBParser, Polypeptide, is_aa
from esm import FastaBatchedDataset, pretrained
from tqdm import tqdm
import os
import numpy as np
from pathlib import Path

CREATE EMBEDDINGS FOR PDB FILES AND STORE THEM

In [ ]:
def get_seq_aa(pdb_file, chain_id='A'):
    chain = PDBParser(QUIET=True).get_structure(str(pdb_file.stem), str(pdb_file))[0][chain_id]
    aa_residues = []
    seq = ""
    for residue in chain.get_residues():
        aa = residue.get_resname()
        if not is_aa(aa) or not residue.has_id('CA'): # Not amino acid
            continue
        elif aa == "UNK":  # unknown amino acid
            seq += "X"
        else:
            seq += Polypeptide.three_to_one(residue.get_resname())
        aa_residues.append(residue)
    return seq, aa_residues

In [ ]:
CHAIN_ID = 'H'

# ONLY EMBEDDINGS - NOT CONTACT MAPS
def extract_embeddings(model_name, pdb_dir, output_dir, repr_layers=[6]):
    """   GETS A DIRECTORY OF .pdb FILES, USES AN esm MODEL TO CREATE
          EMBEDDINGS FOR A SPECIFIC CHAIN (CHAIN_ID) IN EACH FILE, AND STORES
          A .pt FILE WITH AN EMBEDDING VECTOR FOR THAT CHAIN   """

    model, alphabet = pretrained.load_model_and_alphabet(model_name)
    print("done download")
    model.eval()

    if torch.cuda.is_available():
        model = model.cuda()
    output_dir.mkdir(parents=True, exist_ok=True)

    pdb_dir = pathlib.Path(pdb_dir)
    pdb_files = list(pdb_dir.glob('*.pdb'))

    with torch.no_grad():
        for pdb_file in pdb_files:
            seq, _ = get_seq_aa(pdb_file, CHAIN_ID)

            batch_converter = alphabet.get_batch_converter()
            batch_labels, batch_strs, batch_tokens = batch_converter([(str(pdb_file.stem), seq)])

            if torch.cuda.is_available():
                batch_tokens = batch_tokens.to(device="cuda", non_blocking=True)

            out = model(batch_tokens, repr_layers=repr_layers, return_contacts=False)

            representations = {layer: t.to(device="cpu") for layer, t in out["representations"].items()}

            for i, label in enumerate(batch_labels):
                entry_id = label.split()[0]

                filename = output_dir / f"{entry_id}.pt"
                result = {"entry_id": entry_id}

                # save amino acid embeddings instead of mean representation
                result["amino_acid_embeddings"] = {layer: t[i, 1:-1].clone() for layer, t in representations.items()}
                torch.save(result, filename)


model_name = 'esm2_t6_8M_UR50D' # esm2_t36_3B_UR50D 'esm1b_t33_650M_UR50S'
pdb_dir = '/content/drive/MyDrive/Colab Notebooks/Ex4Data/'
output_dir = './hackathon_output/'

extract_embeddings(model_name, pdb_dir, pathlib.Path(output_dir))

READ THE EMBEDDING OF A SPECIFIC PROTEIN. THE PRINTED DATA IS A 2D TENSOR WITH SHAPE: [NUM_AMINO_ACIDS, EMBEDDING_LENGTH].


In [ ]:

# # THIS IS ACTUALLY AN EMBEDDING OF A CHAIN. CHANGE THE EMBEDDING FUNCTION TO GET FOR WHOLE PROTEIN.
# filename = '/content/hackathon_output/12E8_1.pt'

# data = torch.load(filename)
# print(data['amino_acid_embeddings'][33])
# #print(len(data['mean_representations'][33]))

In [ ]:
def generate_input(pt_file):
  data = torch.load(pt_file)['amino_acid_embeddings'][6]  # TODO - CHANGE BY NUM OF LAYERS OF esm MODEL
  padded_data= torch.zeros((140, 320))  # TODO - CHANGE BY EMBEDDING DIMENTION OF esm MODEL
  padded_data[:data.size(0),:] = data
  return padded_data.numpy()

In [ ]:
NB_CHAIN_ID = "H"
NB_MAX_LENGTH = 140
AA_DICT = {"A": 0, "C": 1, "D": 2, "E": 3, "F": 4, "G": 5, "H": 6, "I": 7, "K": 8, "L": 9, "M": 10, "N": 11,
           "P": 12, "Q": 13, "R": 14, "S": 15, "T": 16, "W": 17, "Y": 18, "V": 19, "X": 20, "-": 21}
FEATURE_NUM = len(AA_DICT)
BACKBONE_ATOMS = ["N", "CA", "C", "O", "CB"]
OUTPUT_SIZE = len(BACKBONE_ATOMS) * 3


def generate_label(pdb_file):  # TODO: implement this!
    """
    receives a pdb file and returns its backbone + CB coordinates.
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H') already alingned to a reference nanobody.
    :return: numpy array of shape (CDR_MAX_LENGTH, OUTPUT_SIZE).
    """
    print(pdb_file)
    # Get the sequence of amino acid residues
    _, aa_residues = get_seq_aa(pdb_file, NB_CHAIN_ID)

    # Create an empty numpy array of shape (140,15) filled with zeroes
    coords_matrix = np.zeros((NB_MAX_LENGTH, OUTPUT_SIZE))

    # Iterate over the residues and populate the coords_matrix
    for i in range(len(aa_residues)):
        residue = aa_residues[i]
        for j in range(len(BACKBONE_ATOMS)):
            atom = BACKBONE_ATOMS[j]
            without_gly = atom in residue and residue.get_resname() != "GLY"
            with_gly = atom in residue and residue.get_resname() == "GLY" and j != "CB"
            if without_gly or with_gly:  # Glycine doesn't have a CB atom
                coords_matrix[i, j*3:j*3+3] = residue[atom].get_coord()
            else:
                coords_matrix[i, j*3:j*3+3] = (0, 0, 0)

    return coords_matrix

CREATE INPUT FOR NEURAL NET. THIS MATRIX REPRESENTS ALL PROTEINS IN DB. EACH ROW IS AN AMINO ACID EMBEDDING.

In [ ]:


input_matrix = []
labels_matrix = []
encodings_data_path = pathlib.Path('./hackathon_output/')  # TODO: change path if needed
pdb_data_path = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/Ex4Data/')

files_in_pdb = list(pdb_data_path.glob('*'))
for protein in files_in_pdb:
  protein_no_extention = protein.stem
  pdb_path = pdb_data_path / (protein_no_extention + protein.suffix)
  embedding_path = encodings_data_path / (protein_no_extention + '.pt')

  encoded_protein = generate_input(embedding_path)  # turn torch tensor to numpy array
  nb_xyz = generate_label(pdb_path)
  input_matrix.append(encoded_protein)
  labels_matrix.append(nb_xyz)

save_path = Path('/content/drive/MyDrive/Colab Notebooks/final_training_matrices')  # convert string to Path object
save_path.mkdir(parents=True, exist_ok=True)
np.save(f"{save_path}/train_input.npy", np.array(input_matrix))
np.save(f"{save_path}/train_labels.npy", np.array(labels_matrix))

print(f"Number of samples: {len(input_matrix)}")


MAKE SURE THAT TRAIN/LABEL MATRICES LOOK OK

In [ ]:

# # Path to your .npy file
# file_path = '/content/drive/MyDrive/Colab Notebooks/final_training_matrices/train_input.npy'

# # Load the data
# data = np.load(file_path, allow_pickle=True)

# # Print the data
# print(data.shape)

(1974, 140, 320)


TEST EMBEDDING + CONTCACT

In [ ]:
!pip install biopython
!pip install torch
!pip install tape_proteins
!pip install Bio
!pip install import-ipynb
!pip install git+https://github.com/facebookresearch/esm.git
import pathlib
import torch
from Bio.PDB import PDBParser, Polypeptide, is_aa
from esm import FastaBatchedDataset, pretrained
from tqdm import tqdm
import os
import numpy as np
from pathlib import Path


In [ ]:
def extract_embeddings_and_contact_maps(model_name, protain_seq, protain_name):
    os.makedirs(utils.path_to_save_emmbending(model_name), exist_ok=True)
    repr_layers = [utils.LAYERS_NUMBER[model_name]]
    model, alphabet = pretrained.load_model_and_alphabet(model_name)
    print("done download")
    model.eval()

    if torch.cuda.is_available():
        model = model.cuda()

    with torch.no_grad():
        batch_converter = alphabet.get_batch_converter()
        batch_labels, batch_strs, batch_tokens = batch_converter([(protain_name, protain_seq)])

        if torch.cuda.is_available():
            batch_tokens = batch_tokens.to(device="cuda", non_blocking=True)

        out = model(batch_tokens, repr_layers=repr_layers, return_contacts=True)

        representations = {layer: t.to(device="cpu") for layer, t in out["representations"].items()}
        contacts = out["contacts"].to(device="cpu")  # New line to extract contact maps

        for i, label in enumerate(batch_labels):
            entry_id = label.split()[0]

            filename = os.path.join(utils.path_to_save_emmbending(model_name), f"{entry_id}.pt")
            result = {"entry_id": entry_id}

            # save amino acid embeddings and contact map instead of mean representation
            result["amino_acid_embeddings"] = {layer: t[i, 1:-1].clone() for layer, t in representations.items()}
            result["contact_map"] = contacts[i]  # Save the contact map

            torch.save(result, filename)

def generate_input_with_contact_maps(pt_file, model_name):
  repr_layers = utils.LAYERS_NUMBER[model_name]
  data = torch.load(pt_file)

  amino_acid_embeddings = data['amino_acid_embeddings'][repr_layers]
  contact_map = data['contact_map']

  # Padding both amino acid embeddings and contact maps
  padded_embeddings = torch.zeros((140, utils.EMBENDING_DIM[model_name]))
  padded_embeddings[:amino_acid_embeddings.size(0),:] = amino_acid_embeddings

  padded_contact_map = torch.zeros((140, 140))  # Assuming contact maps are 2D
  padded_contact_map[:contact_map.size(0), :contact_map.size(1)] = contact_map

  combined_input = np.concatenate((padded_embeddings.numpy(), padded_contact_map.numpy()), axis=1)

  return combined_input


In [ ]:
# # GET EMBEDDINGS + CONTACT MAPS, AND SAVE THEM TO output_dir AS .pt FILES
# def extract_embeddings(model_name, pdb_dir, output_dir, repr_layers=[36]):
#     model, alphabet = pretrained.load_model_and_alphabet(model_name)
#     model.eval()
#     if torch.cuda.is_available():
#         model = model.cuda()
#     output_dir.mkdir(parents=True, exist_ok=True)
#     pdb_dir = pathlib.Path(pdb_dir)
#     pdb_files = list(pdb_dir.glob('*.pdb'))

#     with torch.no_grad():
#         for pdb_file in pdb_files:
#             seq, _ = get_seq_aa(pdb_file, CHAIN_ID)
#             batch_converter = alphabet.get_batch_converter()
#             batch_labels, batch_strs, batch_tokens = batch_converter([(str(pdb_file.stem), seq)])
#             if torch.cuda.is_available():
#                 batch_tokens = batch_tokens.to(device="cuda", non_blocking=True)
#             out = model(batch_tokens, repr_layers=repr_layers, return_contacts=True)
#             representations = {layer: t.to(device="cpu") for layer, t in out["representations"].items()}
#             contact_map = out['contacts'].to(device="cpu")
#             for i, label in enumerate(batch_labels):
#                 entry_id = label.split()[0]
#                 filename = output_dir / f"{entry_id}.pt"
#                 result = {"entry_id": entry_id}
#                 result["amino_acid_embeddings"] = {layer: t[i, 1:-1].clone() for layer, t in representations.items()}
#                 result["contact_map"] = contact_map[i, 1:-1].clone()
#                 torch.save(result, filename)

# model_name = 'esm2_t36_3B_UR50D' # esm2_t36_3B_UR50D 'esm1b_t33_650M_UR50S'
# pdb_dir = '/content/drive/MyDrive/Colab Notebooks/Ex4Data/'
# output_dir = '/content/drive/MyDrive/Colab Notebooks/embeddings_for_contacts/'
# extract_embeddings(model_name, pdb_dir, pathlib.Path(output_dir))